<a href="https://colab.research.google.com/github/beppezampieri/SPAZIO-DI-LAVORO/blob/master/Copia_di_Esercizi_9_Soluzione.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SOLUZIONE ESERCIZI LEZIONE 9 - TRIS
---
# Python al Liceo Scientifico Enrico Fermi - edizione 22/23

Davide Bresolin<br>


# Il gioco del Tris

Il tris è un popolarissimo gioco di carta che gioca su una griglia quadrata di 3×3 caselle. 

![Esempio di partita a tris](https://upload.wikimedia.org/wikipedia/commons/f/f6/Tic_Tac_Toe.png)

Negli esercizi che seguono useremo i widget per programmare il gioco del tris in Python. Prima di iniziare importiamo il modulo `ipywidget` e la funzione `display`.

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Esercizio 1

Crea una griglia 3x3 con nove bottoni che sarà l'area di gioco. All'inzio, l'etichetta dei bottoni deve essere vuota.

In [ ]:
for i in range(3):
  riga = []
  for j in range(3):
    btn = widgets.Button(description="")
    riga.append(btn)
  display(widgets.HBox(riga))


Questa soluzione usa due cicli `for` innestati per costruire la griglia. Il ciclo più interno costruisce una riga della griglia, riempiendo la lista `riga` un bottone alla volta. Il ciclo più esterno visualizza le righe una alla volta.

Usare due `for` ci consente di scrivere del codice che è facilmente genralizzabile: se per esempio volessimo creare una griglia 5x5 ci basterebbe sostituire `range(3)` con `range(5)` nelle guardie dei `for`.

# Esercizio 2

Aggiungi un gestore dell'evento *click* ai bottoni, che marca con l'etichetta `X` un bottone che viene premuto. Se il bottone è già marcato l'etichetta `X` deve rimanere inalterata.

In [ ]:
def gestore_evento(b):
  b.description="X"

for i in range(3):
  riga = []
  for j in range(3):
    btn = widgets.Button(description="")
    btn.on_click(gestore_evento)
    riga.append(btn)
  display(widgets.HBox(riga))

Per risolvere questo esercizio ci basta definire un gestore degli eventi che cambia la descrizione del bottone impostandola a `'X'`. La riga `btn.on_click(gestore_evento)` nel ciclo for più interno associa il gestore a tutti bottoni.

# Esercizio 3

Modifica il gestore dell'evento *click* in modo da gestire i turni dei due giocatori. Il primo bottone che viene premuto va marcato con `X`, il secondo con `O`, il terzo con `X`, e così via. Un bottone che è già stato marcato non viene modificato.

In [ ]:
turno = 'X'

def gestore_evento(b):
  global turno
  if b.description == "":
    b.description=turno
    if turno == 'X':
      turno = 'O'
    else:
      turno = 'X'

for i in range(3):
  riga = []
  for j in range(3):
    btn = widgets.Button(description="")
    btn.on_click(gestore_evento)
    riga.append(btn)
  display(widgets.HBox(riga))

Per risolvere questo esercizio dobbiamo aggiungere una variabile `turno` al programma. Questa variabile alterna tra i valori `X` e `O`, e rappresenta il giocatore a cui tocca giocare in ogni momento. 

Il gestore degli eventi si comporta come segue:
- controlla che il bottone non sia già marcato (`if b.description == "":`)
- se il bottone non è marcato, aggiorna la descrizione del bottone con il simbolo memorizzato in `turno`
- aggiorna `turno`, facendolo alternare tra `X` e `O`

Notate che la variabile `turno` è definita prima e al di fuori della funzione `gestore_evento`. Queste variabili sono dette anche **globali** perché ad esse possono accedere tutte le funzioni. A differenza delle variabili locali, che sono distrutte una volta terminata l’esecuzione della loro funzione, quelle globali persistono tra una chiamata di funzione e l’altra e  possono essere usate per memorizzare dell'informazione che si deve mantenere e riutilizzare nel tempo, come il turno del gioco. 

La dichiarazione `global turno` all'inzio del codice di `gestore_evento` serve per dire a Pyhton una cosa del genere: 

> In questa funzione, quando dico `turno`, intendo la variabile globale che ho definito prima: non crearne una nuova. 

Senza questa dichiarazione Python considererebbe `turno` come una nuova variabile, diversa da quella globale definita fuori dalla funzione.

# Esercizio 4

Modifica il gestore dell'evento *click* in modo che controlli se uno dei due giocatori ha vinto. Quando uno dei due giocatori riesce a disporre tre dei propri simboli in linea retta orizzontale, verticale o diagonale, il programma deve stampare `Il giocatore X ha vinto!` oppure `Il giocatore O ha vinto!`. Se la griglia viene riempita senza che nessuno dei giocatori sia riuscito a completare una linea retta di tre simboli, il programma deve stampare `Partita patta!`

## Soluzione

Vediamo la soluzione di questo esercizio per passi successivi. Come prima cosa abbiamo bisogno di altre due variabili globali, oltre al `turno`:
- una variabile `mosse` che ci dirà quante mosse sono state fatte, e che ci servirà per capire se la partita è terminata perché sono state fatte esattamente 9 mosse (e tutte le caselle della griglia 3x3 sono state marcate).
- una variable `arena` che rappresenterà la griglia 3x3 con i simboli che sono stati scritti nelle celle. All'inzio `arena` è una lista che contiene 3 liste (una per riga), ognuna delle quali contiene tre stringhe vuote:
```python
[["", "", ""], 
["", "", ""],
["", "", ""]]
```
Per accedere al simbolo che si trova nella cella nella posizione `riga`, `colonna` dobbiamo usare due volte l'operatore `[ ]`:
```python
arena[riga][colonna]
```

In [ ]:
turno = 'X'
mosse = 0
arena = []
for i in range(3):
  riga = []
  for j in range(3):
    riga.append("")
  arena.append(riga)

Ora che abbiamo le variabili globali che ci servono, possiamo definire una funzione ausiliaria `check` che prende tre argomenti: l'`arena` di gioco, e la posizione `riga`, `colonna` dell'ultimo simbolo marcato, e controlla se il giocatore che ha messo l'ultimo simbolo ha vinto oppure no. In caso positivo ritorna `True`, altrimenti ritorna `False`.

La funzione esegue tre controlli:
- controlla se i simboli nella riga `riga` sono tutti uguali, e ritorna `True` se questo è il caso;
- controlla se i simboli nella colonna `colonna` sono tutti uguali, e ritorna `True` se lo sono;
- controlla le diagonali se la posizione `riga`, `colonna` si trova su una o entrabe le diagonali;
- infine, se nessuno dei controlli è andato a buon fine, ritorna `False`.

In [ ]:
def check(arena, riga, colonna):
  mark = arena[riga][colonna]
  # check riga
  vinto = True
  for i in range(3):
    if arena[riga][i] != mark:
      vinto = False
  if vinto:
    return True
  # check colonna
  vinto = True
  for i in range(3):
    if arena[i][colonna] != mark:
      vinto = False
  if vinto:
    return True
  # check diagonale
  if riga == colonna:
    vinto = True
    for i in range(3):
      if arena[i][i] != mark:
        vinto = False
    if vinto:
        return True
  if riga + colonna == 2:
    vinto = True
    for i in range(3):
      if arena[i][2-i] != mark:
        vinto = False
    if vinto:
      return True
  return False

Il nuovo gestore degli eventi, dopo aver marcato una nuova cella del gioco chiama `check` per vedere se il giocatore corrente ha vinto oppure no. 
Quando il numero di mosse arriva a 9, e se l'ultimo giocatore che gioca non ha vinto, dichiara la partita patta. Per capire in quale posizione della griglia è stato messo il simbolo, la funzione usa gli attributi `b.riga` e `b.colonna` del bottone che è stato premuto. Questi attributi saranno creati dal codice che visualizza la griglia 3x3, come mostrato nella cella seguente.

In [ ]:
def gestore_evento(b):
  global turno, arena, mosse
  if b.description == "":
    b.description=turno
    arena[b.riga][b.colonna] = turno
    mosse = mosse + 1
    if check(arena, b.riga, b.colonna):
      print("Il giocatore", turno, "ha vinto!")
    elif mosse >= 9:
      print("Partita patta!")
    if turno == 'X':
      turno = 'O'
    else:
      turno = 'X'

Ultimo passo del programma è la creazione e visualizzazione dei bottoni. Il codice con i due `for` innestati visto negli esercizi precedenti viene aggiornato con due nuovi comandi:
```python
btn.riga = i
btn.colonna = j
```
che creano due nuovi attributi di un bottone: `riga` e `colonna`, e che memorizzano la posizione del bottone nella griglia.

In [ ]:
for i in range(3):
  riga = []
  for j in range(3):
    btn = widgets.Button(description="")
    btn.on_click(gestore_evento)
    btn.riga = i
    btn.colonna = j
    riga.append(btn)
  display(widgets.HBox(riga))

Partita patta!


# Estensioni e varianti

Ora che hai scritto un programma per giocare a tris, puoi provare a migliorarlo o a scrivere un programma per giocare ad un gioco più complesso.

Alcuni esempi di migliorie che puoi provare a fare sono:

- Abbellisci la grafica e la disposizione dei bottoni
- Modifica il programma in modo che il giocatore `O` sia il computer
- Gioca su una scacchiera 5x5 o 7x7. In questo caso vince il primo giocatore che riesce a disporre **quattro** simboli in fila.

Altri giochi che si giocano su una griglia e che puoi provare a fare sono:

- [Forza 4](https://it.wikipedia.org/wiki/Forza_quattro) è un gioco di allineamento dove l'obiettivo è mettere in fila (orizzontale, verticale o diagonale) quattro dell proprie pedine. L'elemento fondamentale del gioco, che lo rende del tutto originale, è la forza di gravità: le pedine vengono fatte cadere lungo una griglia verticale, in modo tale che una pedina inserita in una certa colonna vada sempre a occupare la posizione libera situata più in basso nella colonna stessa.

- [Esapedone](https://it.wikipedia.org/wiki/Esapedone) è un gioco che si svolge su una scacchiera 3x3 su cui sono collocati sei pedoni, tre bianchi e tre neri. Ogni pedone si muove seguendo le regole degli scacchi: può avanzare di una casella o catturare diagonalmente un pezzo avversario. Il gioco si vince raggiungendo la riga di partenza dell'avversario con un proprio pedone, catturando tutti i pedoni avversari, oppure quando l'avversario non può muovere al suo turno perché i propri pedoni sono bloccati; non è possibile la patta.